Nội dung Lý Thuyết:

- Hệ Phân Tán (Distributed System)
- Concurency vs Paralism
- MapReduce Framework
- Spark Framework Features: `In-memory`, `Lazy`, `Fault-Tolerance`
- RDD (Resilient Distributed Dataset)
- DataFrame, Compare with RDD

**CHÚ Ý:**
Các Bài tập dưới đây sẽ sử dụng file `1M.json` để làm. Nếu bạn chưa upload file `1M.json` vào folder `dbfs:/Filestore/wavepoint`, hãy đọc file hướng dẫn upload file tại Classroom và chú ý kiểm tra xem đã bật `DBFS File Browser` trong `Settings` hay chưa

thực hiện load file sử dụng spark tại `dbfs:/Filestore/wavepoint/1M.json`

In [0]:
val path = "dbfs:/FileStore/wavepoint/1M.json"

val input = spark
  .read
  .json(path)

path1m: String = dbfs:/FileStore/wavepoint/1M.json
input1m: org.apache.spark.sql.DataFrame = [MaxValidSpeed: double, ProviderId: bigint ... 13 more fields]

sử dụng hàm `display()` để hiển thị kết quả, **Lưu ý**: hàm `display()` chỉ chạy được trên Databricks và hỗ trợ hiển thị kết quả và visualize dữ liệu 

In [0]:
display(input.limit(10))

MaxValidSpeed,ProviderId,TimeToBeginAck,TimeToEndAck,Vss,_id,datetime,driver,heading,isOverSpeed,isOverSpeed2,speed,vehicle,x,y
120.0,1028,List(1563753600),593.7526,0.0,List(5d349a13c90a9713bcde22d8),1563753660,,8.0,false,false,0.0,51C70283,106.57334,10.8309183333
120.0,1028,List(1563753600),593.7526,0.0,List(5d349a13c90a9713bcde22da),1563753660,,156.0,false,false,0.0,77H9208,109.0815433333,13.920381666699999
120.0,1028,List(1563753600),593.7526,0.0,List(5d349a13c90a9713bcde22dd),1563753660,,0.0,false,false,0.0,51C85808,106.60483,10.8669633333
120.0,1028,List(1563753600),593.7526,0.0,List(5d349a13c90a9713bcde22de),1563753660,,190.0,false,false,0.0,43B04228,108.2146933333,16.03246
120.0,1028,List(1563753600),609.376,0.0,List(5d349a13c90a9713bcde22df),1563753660,,310.0,false,false,0.0,78C08038,109.2789083333,13.331095
120.0,1028,List(1563753600),609.376,0.0,List(5d349a13c90a9713bcde22e5),1563753660,,60.0,false,false,0.0,30A12576,105.7655333333,21.0317433333
120.0,1028,List(1563753600),609.376,0.0,List(5d349a13c90a9713bcde22ee),1563753660,,86.0,false,false,0.0,51C05422,106.6672483333,10.808566666699999
120.0,1028,List(1563753600),609.376,0.0,List(5d349a13c90a9713bcde22f1),1563753660,,194.0,false,false,0.0,29H21968,106.1326116667,20.619305
120.0,1028,List(1563753600),625.0032,0.0,List(5d349a13c90a9713bcde22f4),1563753660,,290.0,false,false,0.0,77A13695,106.6120583333,10.8371
120.0,1028,List(1563753600),625.0032,0.0,List(5d349a13c90a9713bcde22fb),1563753660,,116.0,false,false,0.0,43B03204,108.1739366667,16.0596383333


Câu 1: Có bao nhiêu bản ghi có `speed > 0` ?

In [0]:
val lesson1 = 

display(lesson1)

Câu 2: Có bao nhiêu ProviderId duy nhất trong `input`

In [0]:
val lesson2 =

display(lesson2)

Câu 3: Hãy lọc biển số xe của tỉnh và đếm số lượng xe của mỗi tỉnh (biển số xe tỉnh là 2 số đầu của trường `vehicle` VD: Với xe có biển số: `29A12345` thì số `29` là biển số của tỉnh)

In [0]:
val lesson3 =

display(lesson3)

Câu 4: Hãy lấy giá trị kinh độ `lon` (`x`) và vĩ độ `lat`(`y`) và sử dụng để gọi tới nominatimAPI để lấy giá trị `display_name` từ API và hiển thị ra kết quả. Đầu ra mong muốn (`x`, `y`, `display_name`). **Chú ý lấy limit 100 bản ghi**

In [0]:
%sh
curl --location 'https://nominatim.openstreetmap.org/reverse?format=json&lat=10.8309183333&lon=106.57334'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   811  100   811    0     0   3975      0 --:--:-- --:--:-- --:--:--  3975
{"place_id":238777979,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"way","osm_id":32576575,"lat":"10.83123646694974","lon":"106.5733607607416","class":"highway","type":"tertiary","place_rank":26,"importance":0.05340036697947658,"addresstype":"road","name":"Thới Hòa","display_name":"Thới Hòa, Khu công nghiệp Vĩnh Lộc, Xã Vĩnh Lộc A, Huyện Bình Chánh, Thành phố Hồ Chí Minh, 71821, Việt Nam","address":{"road":"Thới Hòa","industrial":"Khu công nghiệp Vĩnh Lộc","village":"Xã Vĩnh Lộc A","city_district":"Huyện Bình Chánh","city":"Thành phố Hồ Chí Minh","ISO3166-2-lvl4":"VN-SG","postcode":"71821","country":"Việt Nam","country_code":"v

In [0]:
val lesson4 =

display(lesson4)